# Data generation for K2-18b models from Bower et al. (2024)

In [ ]:
import logging

import numpy as np
from pathlib import Path

from atmodeller import debug_logger
from atmodeller.constraints import ElementMassConstraint, SystemConstraints, BufferedFugacityConstraint
from atmodeller.thermodata.redox_buffers import IronWustiteBuffer
from atmodeller.core import GasSpecies, Species
from atmodeller.interior_atmosphere import Planet, InteriorAtmosphereSystem
from atmodeller.initial_solution import InitialSolutionRegressor, InitialSolutionSwitchRegressor, InitialSolutionDict
from atmodeller.utilities import earth_oceans_to_kg
from atmodeller.eos.saxena import get_saxena_eos_models
from atmodeller.eos.holland import get_holland_eos_models

from atmodeller.eos.holley import get_holley_eos_models
from atmodeller.solubility.hydrogen_species import H2_basalt_hirschmann, H2O_basalt_mitchell
from atmodeller.solubility.carbon_species import CO2_basalt_dixon, CO_basalt_yoshioka, CH4_basalt_ardia
from atmodeller.solubility.other_species import N2_basalt_libourel

logger = debug_logger()
logger.setLevel(logging.INFO)

Parameters for the simulations

In [ ]:
surface_temperature = 2000
number_of_realisations = 5000
training_steps = 250
k218b_planet_mass = 5.15211E25
k218b_surface_radius = 1.6647E7

In [ ]:
def monte_carlo(interior_atmosphere: InteriorAtmosphereSystem, O2_g: GasSpecies,
                number_of_realisations:int=100, reduced: bool|None = None):
    """Monte Carlo driver
    
    Args:
        interior_atmosphere: An interior-atmosphere system
        O2_g: The O2 gas species
        number_of_realisation: Number of simulations to perform
        reduced: If True, only solve for reduced cases. If False, only solve for oxidised cases.
            Otherwise, solve for all. Defaults to None, meaning solve for all.
    """

    # Parameters are normally distributed between bounds.
    # C/H and N/H ratios explore a range up to 2X BSE values.
    # The total H mass explores a range similar to Shorttle et al. (2024).
    number_ocean_moles = np.random.uniform(1, 3500, number_of_realisations)
    ch_ratios = np.random.uniform(0.05, 3.25, number_of_realisations)
    nh_ratios = np.random.uniform(0.001, 0.015, number_of_realisations)

    fo2_min: float = -5
    fo2_max: float = 5
    if reduced is True:
        fo2_max = 0
    elif reduced is False:
        fo2_min = 0
    fo2_shifts = np.random.uniform(fo2_min, fo2_max, number_of_realisations)

    for realisation in range(number_of_realisations):

        mass_H = earth_oceans_to_kg(number_ocean_moles[realisation])
        mass_C = ch_ratios[realisation] * mass_H
        mass_N = nh_ratios[realisation] * mass_H
        constraints = SystemConstraints([
            ElementMassConstraint("H", mass_H),
            ElementMassConstraint("C", mass_C),
            ElementMassConstraint("N", mass_N),
            BufferedFugacityConstraint(O2_g, IronWustiteBuffer(log10_shift=fo2_shifts[realisation], 
                evaluation_pressure=1))
        ])

        # Extra quantities to write to the output
        # For example, it's often helpful to have the constraints expressed in a more convenient
        # form for analysis and plotting.
        extra = {'fO2_shift': fo2_shifts[realisation], 'C/H ratio': ch_ratios[realisation], 
                 'N/H ratio': nh_ratios[realisation], 
                 'Number of ocean moles': number_ocean_moles[realisation]}

        interior_atmosphere.solve(constraints, extra_output=extra, factor=0.1, tol=1e-5, errors="ignore")

Planet properties

In [ ]:
k218b = Planet(surface_temperature=surface_temperature, planet_mass=k218b_planet_mass, surface_radius=k218b_surface_radius)

Set paths

In [ ]:
k218b_ideal_no_sols_training_path = Path(f"k218b_{surface_temperature}K_ideal_no_sols_{training_steps}its")
k218b_ideal_no_sols_path = Path(f"k218b_{surface_temperature}K_ideal_no_sols_{number_of_realisations}its")
k218b_ideal_with_sols_training_path = Path(f"k218b_{surface_temperature}K_ideal_with_sols_{training_steps}its")
k218b_ideal_with_sols_path = Path(f"k218b_{surface_temperature}K_ideal_with_sols_{number_of_realisations}its")
k218b_real_red_with_sols_training_path = Path(f"k218b_{surface_temperature}K_real_red_with_sols_{training_steps}its")
k218b_real_red_with_sols_path = Path(f"k218b_{surface_temperature}K_real_red_with_sols_{number_of_realisations}its")
k218b_real_oxi_with_sols_training_path = Path(f"k218b_{surface_temperature}K_real_oxi_with_sols_{training_steps}its")
k218b_real_oxi_with_sols_path = Path(f"k218b_{surface_temperature}K_real_oxi_with_sols_{number_of_realisations}its")

## Ideal gas

In [ ]:
H2O_g = GasSpecies("H2O")
H2_g = GasSpecies("H2")
O2_g = GasSpecies("O2")
CO_g = GasSpecies("CO")
CO2_g = GasSpecies("CO2")
CH4_g = GasSpecies("CH4")
N2_g = GasSpecies("N2")
NH3_g = GasSpecies("NH3")

species = Species([H2O_g, H2_g, O2_g, CO_g, CO2_g, CH4_g, N2_g, NH3_g])

### No solubilities

Start with a dictionary of initial guesses and then switch to a regressor to provide a training dataset.

In [ ]:
values = {H2_g: 1000.0, H2O_g: 1000.0}
initial_solution_start = InitialSolutionDict(values, species=species)
initial_solution_training = InitialSolutionSwitchRegressor(initial_solution_start, species=species, fit=True, fit_batch_size=50, partial_fit=True, partial_fit_batch_size=100)

Now run this training model and save the output to a pickle file.

In [ ]:
k218b_ideal_no_sols_training = InteriorAtmosphereSystem(species=species, initial_solution=initial_solution_training, planet=k218b)
monte_carlo(k218b_ideal_no_sols_training, O2_g=O2_g, number_of_realisations=training_steps)
k218b_ideal_no_sols_training.output(file_prefix=k218b_ideal_no_sols_training_path, to_pickle=True, to_excel=True);

Now, we can use the training output generated above to train a network and use this to improve convergence and speed up the computation of the final run.

In [ ]:
initial_solution_full = InitialSolutionRegressor.from_pickle(k218b_ideal_no_sols_training_path.with_suffix(".pkl"), species=species, fit=False, partial_fit=True, partial_fit_batch_size=1000)

Run the full model and export the data.

In [ ]:
k218b_ideal_no_sols = InteriorAtmosphereSystem(species=species, initial_solution=initial_solution_full, planet=k218b)
monte_carlo(k218b_ideal_no_sols, O2_g=O2_g, number_of_realisations=number_of_realisations)
k218b_ideal_no_sols.output(file_prefix=k218b_ideal_no_sols_path, to_pickle=True, to_excel=True)

### With solubilities

This will set all the solubilities (where possible) to conform to a basaltic composition. Note that once this is set the solubilities are always applied to the planet.

In [ ]:
k218b.melt_composition = 'basalt'

The pickle file output for the training model without solubilities (above) is used to inform the initial condition for the training set with solubilities, but because solubilities can strongly affect the solution, we use the `species_fill` dictionary to modify some initial guesses.

In [ ]:
species_fill = {H2O_g: 50}
initial_solution_training = InitialSolutionRegressor.from_pickle(k218b_ideal_no_sols_training_path.with_suffix(".pkl"), species=species, species_fill=species_fill, fit=True, fit_batch_size=50, partial_fit=True, partial_fit_batch_size=25)

Run the training model and save the output to a pickle file.

In [ ]:
k218b_ideal_with_sols_training = InteriorAtmosphereSystem(species=species, initial_solution=initial_solution_training, planet=k218b)
monte_carlo(k218b_ideal_with_sols_training, O2_g=O2_g, number_of_realisations=training_steps)
k218b_ideal_with_sols_training.output(file_prefix=k218b_ideal_with_sols_training_path, to_pickle=True, to_excel=True);

Now, we can use the training output generated above to train a network and use this to improve convergence and speed up the computation of the final run.

In [ ]:
initial_solution_full = InitialSolutionRegressor.from_pickle(k218b_ideal_with_sols_training_path.with_suffix(".pkl"), species=species, fit=False, partial_fit=True, partial_fit_batch_size=1000)

Run the full model and export the data.

In [ ]:
k218b_ideal_with_sols = InteriorAtmosphereSystem(species=species, initial_solution=initial_solution_full, planet=k218b)
monte_carlo(k218b_ideal_with_sols, O2_g=O2_g, number_of_realisations=number_of_realisations)
k218b_ideal_with_sols.output(file_prefix=k218b_ideal_with_sols_path, to_pickle=True, to_excel=True)

## Real gas

We redefine the planet to reset the solubility laws applied for the ideal gas case with solubilities.

In [ ]:
k218b = Planet(surface_temperature=surface_temperature, planet_mass=k218b_planet_mass, surface_radius=k218b_surface_radius)

The solubilities are set below manually to use the Mitchell H2O law, which is not the default law for basaltic compositions in the composition dictionary applied for ideal cases.

In [ ]:
eos_holland = get_holland_eos_models() 
eos_saxena = get_saxena_eos_models()
eos_holley = get_holley_eos_models()

H2O_g = GasSpecies("H2O", solubility=H2O_basalt_mitchell(), eos=eos_holland["H2O"])
H2_g = GasSpecies("H2", solubility=H2_basalt_hirschmann(), eos=eos_saxena["H2"])
O2_g = GasSpecies("O2")
CO_g = GasSpecies("CO", solubility=CO_basalt_yoshioka(), eos=eos_holland["CO"])
CO2_g = GasSpecies("CO2", solubility=CO2_basalt_dixon(), eos=eos_holland["CO2"])
CH4_g = GasSpecies("CH4", solubility=CH4_basalt_ardia(), eos=eos_holland["CH4"])
N2_g = GasSpecies("N2", solubility=N2_basalt_libourel())
NH3_g = GasSpecies("NH3")

species: Species = Species([H2O_g, H2_g, O2_g, CO_g, CO2_g, CH4_g, N2_g, NH3_g])

### With solubilities

#### Reduced cases

In [ ]:
values = {H2_g: 1000, H2O_g: 10, CO2_g:10, CO_g:100, CH4_g:100, N2_g:1, NH3_g:10}
initial_solution_start = InitialSolutionDict(value=values, species=species)

Run the training model and save the output to a pickle file.

In [ ]:
k218b_real_red_with_sols_training = InteriorAtmosphereSystem(species=species, initial_solution=initial_solution_start, planet=k218b)
monte_carlo(k218b_real_red_with_sols_training, O2_g=O2_g, number_of_realisations=training_steps, reduced=True)
k218b_real_red_with_sols_training.output(file_prefix=k218b_real_red_with_sols_training_path, to_pickle=True, to_excel=True);

See how many solves failed (but these are ignored by default).

In [ ]:
k218b_real_red_with_sols_training.failed_solves;

Now, we can use the training output generated above to train a network and use this to improve convergence and speed up the computation of the final run.

In [ ]:
initial_solution_full = InitialSolutionRegressor.from_pickle(k218b_real_red_with_sols_training_path.with_suffix(".pkl"), species=species, fit=False, partial_fit=True, partial_fit_batch_size=1000)

Run the full model and export the data.

In [ ]:
k218b_real_red_with_sols = InteriorAtmosphereSystem(species=species, initial_solution=initial_solution_full, planet=k218b)
monte_carlo(k218b_real_red_with_sols, O2_g=O2_g, number_of_realisations=number_of_realisations, reduced=True)
k218b_real_red_with_sols.output(file_prefix=k218b_real_red_with_sols_path, to_pickle=True, to_excel=True)

Report the number of failed solves.

In [ ]:
k218b_real_red_with_sols.failed_solves;

#### Oxidised cases

In [ ]:
values = {H2_g: 10, H2O_g: 100, CO2_g:1000, CO_g:100, CH4_g:10, N2_g:1, NH3_g:1}
initial_solution_start = InitialSolutionDict(value=values, species=species)

Run the training model and save the output to a pickle file.

In [ ]:
k218b_real_oxi_with_sols_training = InteriorAtmosphereSystem(species=species, initial_solution=initial_solution_start, planet=k218b)
monte_carlo(k218b_real_oxi_with_sols_training, O2_g=O2_g, number_of_realisations=training_steps, reduced=False)
k218b_real_oxi_with_sols_training.output(file_prefix=k218b_real_oxi_with_sols_training_path, to_pickle=True, to_excel=True);

See how many solves failed (but these are ignored by default).

In [ ]:
k218b_real_oxi_with_sols_training.failed_solves;

Now, we can use the training output generated above to train a network and use this to improve convergence and speed up the computation of the final run.

In [ ]:
initial_solution_full = InitialSolutionRegressor.from_pickle(k218b_real_oxi_with_sols_training_path.with_suffix(".pkl"), species=species, fit=False, partial_fit=True, partial_fit_batch_size=1000)

Run the full model and export the data.

In [ ]:
k218b_real_oxi_with_sols = InteriorAtmosphereSystem(species=species, initial_solution=initial_solution_full, planet=k218b)
monte_carlo(k218b_real_oxi_with_sols, O2_g=O2_g, number_of_realisations=number_of_realisations, reduced=False)
k218b_real_oxi_with_sols.output(file_prefix=k218b_real_oxi_with_sols_path, to_pickle=True, to_excel=True)

Report the number of failed solves.

In [ ]:
k218b_real_oxi_with_sols.failed_solves;